In [2]:
sc.addPyFile("../graphframes-0.8.1-spark3.0-s_2.12.jar")

In [3]:
from graphframes import *
from pyspark.sql.functions import *

In [4]:
# Vertics DataFrame
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 37),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 38),
  ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend"),
  ("g", "e", "follow")
], ["src", "dst", "relationship"])

# Create a GraphFrame
g = GraphFrame(v, e)

g.vertices.show()
g.edges.show()

Py4JJavaError: An error occurred while calling o87.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:466)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:563)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:496)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:564)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.

In [ ]:
#Q1
twohop = g.find("(a)-[]->(b); (b)-[]->(c)")\
        .filter('a.name == "Alice" AND a.id != c.id')\
        .select('c.name').show()


In [ ]:
#Q2
twohop = g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(d)")\
        .filter('a.name == "Alice" AND a.id != c.id AND a.id != d.id')\
        .select('c.name').show()

In [22]:
#Q3
followCharlie = g.find("(a)-[e]->(b)")\
        .filter("e.relationship = 'follow'")\
        .filter('b.name == "Charlie"')\
        .select('a.name').show()

+-----+
| name|
+-----+
|Fanny|
|  Bob|
+-----+



In [65]:
#Q4
Celebrity = g.find("()-[e]->(a)")\
            .filter("e.relationship = 'follow'")\
            .groupBy('a').count().filter('count>=2')\
            .select('a.name').show()

+-------+
|   name|
+-------+
|Charlie|
+-------+



In [5]:
#Q5
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory. Required for stateful transformations
ssc.checkpoint("checkpoint")

numPartitions = 8
rdd = sc.textFile('adj_noun_pairs.txt', numPartitions)
rddQueue = rdd.randomSplit([1]*10, 123)


lines = ssc.queueStream(rddQueue)

def updateFunc(newValues, running):
    if running == None and len(newValues) != 0:
        return newValues[0]
    elif running != None and len(newValues) == 0:
        return running
    elif running != None and len(newValues) != 0:
        if len(newValues[0]) > len(running):
            return newValues[0]
        else:
            return running

def printre(rdd):
    print(rdd.take(5))
    
def printgood(rdd):
    print("GOOD:",rdd.map(lambda rdd: rdd[1]).take(1))
    print()

pairs  = lines.flatMap(lambda line: [line.split(" ")]).filter(lambda x: len(x) == 2)\
        .map(lambda line: (line[0],line[1]))\
        .reduceByKey(lambda x,y:x if len(x) >= len(y) else y)\
        .updateStateByKey(updateFunc)
pairs.foreachRDD(printre)

good = pairs.filter(lambda x: x[0] == 'good')
good.foreachRDD(printgood)


ssc.start()
ssc.awaitTermination(50)
ssc.stop(False)
print("Finished")

[('social', 'constructionism'), ('coercive', 'deprogramming'), ('individual', 'meaninglessness'), ('political', 'self-determination'), ('desirable', 'characteristic')]
GOOD: ['morning/afternoon']

[('social', 'constructionism'), ('coercive', 'deprogramming'), ('individual', 'meaninglessness'), ('political', 'self-determination'), ('desirable', 'characteristic')]
GOOD: ['morning/afternoon']

[('social', 'constructionism'), ('coercive', 'administration'), ('individual', 'meaninglessness'), ('political', 'assassinations-were'), ('desirable', 'characteristic')]
GOOD: ['morning/afternoon']

[('social', 'constructionism'), ('coercive', 'administration'), ('individual', 'being-in-the-world'), ('political', 'assassinations-were'), ('desirable', 'characteristic')]
GOOD: ['morning/afternoon']

[('social', 'realism/constructivism'), ('coercive', 'administration'), ('individual', 'being-in-the-world'), ('political', 'assassinations-were'), ('desirable', 'characteristic')]
GOOD: ['morning/afternoon